# Lab3 - Computational Intelligence
## N-Puzzle 

In [ ]:
import heapq
import random
import time

### Definition of heuristics

In [ ]:
# Precompute goal positions
def precompute_goal_positions(n):
    # Subtracting 1 aligns the value with zero-based indexing.
    return {value: divmod(value - 1, n) for value in range(1, n * n)}

# Manhattan distance function
def manhattan_distance(state, goal_positions, n):
    """Calculate the Manhattan distance for the given state using precomputed goal positions."""
    distance = 0
    for i in range(n):
        for j in range(n):
            value = state[i][j]
            if value != -1:  # Skip the blank tile
                goal_x, goal_y = goal_positions[value]
                distance += abs(i - goal_x) + abs(j - goal_y)
                # value - 1 divided by n (integer division) gives the row index (goal_x).
                # value - 1 modulo n gives the column index (goal_y).
    return distance

In [ ]:
def linear_conflict(state, goal_positions, n):
    """
    Compute the Linear Conflict heuristic for the given state.
    Two tiles ‘a’ and ‘b’ are in a linear conflict if they are in the same row or column ,
    also their goal positions are in the same row or column and the goal position of one of 
    the tiles is blocked by the other tile in that row.
    """
    conflict = 0
    
    # Check rows for linear conflicts
    for row in range(n):
        max_tile = -1
        for col in range(n):
            value = state[row][col]
            if value != -1:
                goal_x, goal_y = goal_positions[value]
                # Tile is in the correct row
                if goal_x == row:
                    if value > max_tile:
                        max_tile = value
                    else:
                        # Conflict detected
                        conflict += 2

    # Check columns for linear conflicts
    for col in range(n):
        max_tile = -1
        for row in range(n):
            value = state[row][col]
            if value != -1:
                goal_x, goal_y = goal_positions[value]
                # Tile is in the correct column
                if goal_y == col:
                    if value > max_tile:
                        max_tile = value
                    else:
                        # Conflict detected
                        conflict += 2

    return conflict

In [ ]:
def linear_conflict_manhattan(state, goal_positions, n):
    """
    Combined Linear Conflict and Manhattan Distance heuristic.
    """
    return manhattan_distance(state, goal_positions, n) + linear_conflict(state, goal_positions, n)

In [ ]:
def misplaced_tiles(state, goal, n):
    """Misplaced tiles heuristic."""
    return sum(1 for i in range(n) for j in range(n) if state[i][j] != goal[i][j] and state[i][j] != -1)

In [ ]:
def combined_heuristic(state, goal_positions, goal, n, magic_number):
    """
    Combine Manhattan + Linear Conflict and Misplaced Tiles heuristics.
    Magic number controls the probability of favoring misplaced tiles.
    """

    # Randomly decide which heuristic to use based on the magic number
    if magic_number > random.random():
        return misplaced_tiles(state, goal, n)  # Favor exploration
    else:
        return linear_conflict_manhattan(state, goal_positions, n)  # Favor exploitation

### Utils

In [ ]:
def is_solvable(state):
    """
    Determines if a given N x N sliding puzzle is solvable based on:
    - Number of inversions
    - Position of the blank tile
    """
    # Flatten the puzzle state and remove the blank tile (-1)
    flat_state = [num for row in state for num in row if num != -1]
    
    # Count inversions
    inversions = 0
    for i in range(len(flat_state)):
        for j in range(i + 1, len(flat_state)):
            if flat_state[i] > flat_state[j]:
                inversions += 1

    # Determine the size of the grid (N x N)
    n = len(state)

    # Find the row of the blank tile (-1) from the bottom
    blank_row_from_bottom = n - next(i for i, row in enumerate(state) if -1 in row)

    # Apply solvability rules
    if n % 2 == 1:  # For odd-sized grids
        return inversions % 2 == 0
    else:  # For even-sized grids
        if blank_row_from_bottom % 2 == 0:  # Blank on an even row from bottom
            return inversions % 2 != 0
        else:  # Blank on an odd row from bottom
            return inversions % 2 == 0
    


In [ ]:
def generate_random_seed(n, seed):
    """Generate a random solvable N²-1 puzzle state."""
    random.seed(seed)
    numbers = list(range(1, n * n)) + [-1]
    random.shuffle(numbers)
    state = [numbers[i:i + n] for i in range(0, len(numbers), n)]
    return state

In [ ]:
def generate_random_steps(n, random_steps, goal):
    state = [row[:] for row in goal]

    for _ in range(random_steps):

        # Find the empty tile (-1)
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] == -1:
                    x, y = i, j
                    break

        # Compute valide moves (up, down, left, right)
        moves = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
        valid_moves = [(nx, ny) for nx, ny in moves if 0 <= nx < n and 0 <= ny < n]

        # Choose a random action and update the state
        nx, ny = random.choice(valid_moves)
        state[x][y], state[nx][ny] = state[nx][ny], state[x][y]

    return state

In [ ]:
def generate_random_matrix(n, seed, random_steps, goal):
    if seed is not None:
        return generate_random_seed(n, seed)        #generate totally random matrix
    else:
        return generate_random_steps(n, random_steps, goal)              #generate matrix with fixed number of permutations

#### A* adaptive
When you store a tuple in a heap managed by heapq, the comparison to determine the minimum element is done by considering the elements of the tuple in lexicographical order.

In [ ]:
def a_star_adaptive(start, goal):
    n = len(start)
    frontier = []
    visited = set()
    state_to_parent = {}
    total_actions_evaluated = 0

    # Precompute goal positions
    goal_positions = precompute_goal_positions(n)

    # Parameters for adaptive heuristic selection
    magic_number = 1.125 * n**(n-1) # Initial probability
    adaptive_factor = 0.985  # Decay rate for the magic number
    threshold = 0.0785      # Allow exploration to be done, but with low probability

    # Initialize the frontier with the start state (f, state, g, parent)
    heapq.heappush(frontier, (combined_heuristic(start, goal_positions, goal, n, magic_number), start, 0, None))

    while frontier:
        f, current, g, parent = heapq.heappop(frontier)
        total_actions_evaluated += 1

        current_tuple = tuple(tuple(row) for row in current)

        # Skip already visited states
        if current_tuple in visited:
            continue
        visited.add(current_tuple)

        # Track exploration to build the tree
        state_to_parent[current_tuple] = parent

        # Goal test
        if current == goal:
            path = []
            state = current_tuple
            while state is not None:
                path.append([list(row) for row in state])
                state = state_to_parent[state]
            path.reverse()

            quality = len(path) - 1  # Number of actions to solve
            cost = total_actions_evaluated  # Total nodes expanded
            efficiency = quality / cost if cost != 0 else float('inf')

            print(f"Quality (Solution Length): {quality}")
            print(f"Cost (Nodes Evaluated): {cost}")
            print(f"Efficiency (Quality/Cost): {efficiency:.4f}")
            return path

        # Generate neighbors and push them into the frontier
        for i in range(n):
            for j in range(n):
                if current[i][j] == -1:
                    x, y = i, j

        moves = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
        for nx, ny in moves:
            if 0 <= nx < n and 0 <= ny < n:  # Check bounds
                new_state = [row[:] for row in current]
                new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]

                newstate_tuple = tuple(tuple(row) for row in new_state)
                if newstate_tuple not in visited:
                    new_g = g + 1
                    new_h = combined_heuristic(new_state, goal_positions, goal, n, magic_number)
                    heapq.heappush(frontier, (new_g + new_h, new_state, new_g, current_tuple))

        if(magic_number > threshold):
            magic_number *= adaptive_factor  # Gradually decrease exploration probability

    print("No solution found.")
    return None

#### Execution

In [ ]:
# Initial setup
N = 5
SEED = None
RANDOMIZE_STEPS = 200
GOAL = [[i * N + j + 1 if i * N + j + 1 < N * N else -1 for j in range(N)] for i in range(N)]

START = generate_random_matrix(N, SEED, RANDOMIZE_STEPS, GOAL)


"""
You can try using even a defined matrix. Move the variable START outside this comment.
START = [

]
"""

# Print results
print("Initial State:")
for row in START:
    print(row)

if is_solvable(START):
    # Measure start time
    start_time = time.time()

    # Solve the puzzle
    path_states = a_star_adaptive(START, GOAL)

    # Measure end time
    end_time = time.time()

    execution_time = end_time - start_time

    if path_states is not None:
        path_states.reverse()
        for row in path_states[0]:
            print(row)
        print("-" * 20)
    else:
        print("No solution found.")
    print(f"Execution Time: {execution_time:.4f} seconds")
else:
    print("Matrix is not solvable.")